In [1]:
import pandas as pd
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
import datetime
from time import time

from rectools.dataset.interactions import Interactions
from rectools.dataset import Dataset
from rectools import Columns

from implicit.nearest_neighbours import TFIDFRecommender, BM25Recommender

from rectools.models import ImplicitItemKNNWrapperModel, RandomModel, PopularModel
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.model_selection.time_split import TimeRangeSplitter

In [2]:
tqdm.pandas()
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Загрузка, знакомство, подготовка

**Взаимодействия пользователей с фильмами**

In [3]:
data = pd.read_csv('ml-latest/ratings.csv')

In [4]:
data.sample(5)

,userId,movieId,rating,timestamp
11415122,112376,8961,3.5,1587783524
17542436,172255,91529,5.0,1673633902
30777106,300953,3396,4.0,1445889669
21529542,210078,1268,3.5,1495318063
9537537,93779,1263,5.0,1551056756


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33832162 entries, 0 to 33832161
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 1.0 GB


In [6]:
print(
    f'Испльзовано памяти: {data.memory_usage(deep=True).sum() / 1024 / 1024:.2f}mb'
)

Испльзовано памяти: 1032.48mb


In [7]:
#оптимизация хранения данных
data['userId'] = data['userId'].astype('int32')
data['movieId'] = data['movieId'].astype('int32')
data['rating'] = data['rating'].astype('float16')
#изменение хранения дат
data['timestamp'] = pd.to_datetime(data['timestamp'].parallel_apply(
    lambda x: pd.Timestamp(x, unit='s').date()))

In [8]:
print(
    f'Испльзовано памяти: {data.memory_usage(deep=True).sum() / 1024 / 1024:.2f}mb'
)

Испльзовано памяти: 580.77mb


**Названия фильмов и imbdID**

In [9]:
movies_ml = pd.read_csv('ml-latest/movies.csv')

In [10]:
movies_ml.sample(5)

,movieId,title,genres
72644,235033,Silver Jew (2007),Documentary
34846,145716,Patrick Still Lives! (1980),Horror
3004,3097,"Shop Around the Corner, The (1940)",Comedy|Drama|Romance
70707,227258,The Midnight Sky (2020),Drama|Fantasy|Sci-Fi
47603,173661,The Cook (1965),Comedy|Romance


In [11]:
links_ml = pd.read_csv('ml-latest/links.csv')

In [12]:
links_ml.sample(5)

,movieId,imdbId,tmdbId
11331,50585,31516,78318.0
23992,120122,2965842,293491.0
9651,31963,65651,258.0
27800,129907,1671457,76226.0
4878,4983,89283,35201.0


## Постановка задачи и baseline

Используя историю взаимодействий пользователей с объектами создать двухэтапную рекомендательную модель, которая значительно превзойдёт базовую по совокупности метрик Serendipity и MeanInvUserFreq (способность удивлять непопулярными релевантными объектами) на валидации, а также удовлетворит меня в ходе тестирования (выборочный визуальный анализ).

Валидировать будем на данных за последние 42 дня, разбив их на 3 фолда по 14 дней. Реализовывать кросс-валидацию будем средствами библиотеки RecTools (https://github.com/MobileTeleSystems/RecTools). 

Отслеживаемые метрики:

`Recall` - отношение числа релевантных рекомендаций к общему числу взаимодействий пользователя в тестовом периоде.

`Map@20` - средняя точность рекоммендаций с учётом рангов (https://rectools.readthedocs.io/en/latest/api/rectools.metrics.ranking.MAP.html).

`MeanInvUserFreq` - может принимать значения в диапазоне от 0 до бесконечности. Значение 0 означает, что все рекомендации уникальны для каждого пользователя, тогда как бесконечность указывает на полное отсутствие уникальности и повторяющиеся рекомендации у всех пользователей (https://rectools.readthedocs.io/en/latest/api/rectools.metrics.novelty.MeanInvUserFreq.html).

`Serendipity` - принимает значения от 0 до 1, чаще всего интерпретируется как способность удивлять неожиданными (непопулярными) релевантными айтемами (https://rectools.readthedocs.io/en/latest/api/rectools.metrics.serendipity.Serendipity.html).


In [13]:
# Для более эффективного использования сохраним данные о взаимодействиях в классе Interactions
data.columns = [Columns.User, Columns.Item, Columns.Weight, Columns.Datetime]
interactions = Interactions(data)
interactions.df.sample(2)

,user_id,item_id,weight,datetime
19460444,190394,589,4.0,1996-05-30
20140984,196817,203224,4.5,2020-01-31


**Перекрёстная валидация**

Мы будем использовать последние 3 периода по 14 дней. 

In [14]:
n_splits = 3
cv = TimeRangeSplitter(test_size='14D',
                       n_splits=n_splits,
                       filter_cold_users=False,
                       filter_cold_items=True)
cv.get_test_fold_borders(interactions)

[(Timestamp('2023-06-09 00:00:00', freq='14D'),
  Timestamp('2023-06-23 00:00:00', freq='14D')),
 (Timestamp('2023-06-23 00:00:00', freq='14D'),
  Timestamp('2023-07-07 00:00:00', freq='14D')),
 (Timestamp('2023-07-07 00:00:00', freq='14D'),
  Timestamp('2023-07-21 00:00:00', freq='14D'))]

**Метрики**

In [15]:
K_RECOS = 20
metrics = {
    "serendipity": Serendipity(k=K_RECOS),
    "MeanInvUserFreq": MeanInvUserFreq(k=K_RECOS),
    "prec@20": Precision(k=K_RECOS),
    "recall": Recall(k=K_RECOS)
}

**Бейзлайн (рекомендуем популярное)**

`PopularRecommender` будет рекомендовать 20 самых популярных фильмов за последние 60 дней:

In [16]:
import mymodule 

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [17]:
params = {
    'days': 60,
    'item_column': Columns.Item,
    'dt_column': Columns.Datetime,
    'user_column': Columns.User,
    'parallel': True
}

In [18]:
model = mymodule.PopularRecommender(**params)

In [25]:
K_RECOS

20

In [19]:
results_df = mymodule.validation_function(interactions=interactions,
                                            cv=cv,
                                            model=model,
                                            k_recs=K_RECOS,
                                            metrics=metrics,
                                            model_name='PopularModel')

3it [01:12, 24.04s/it]


In [20]:
results_df

serendipity           MeanInvUserFreq             prec@20  \
                    mean       std            mean       std      mean   
model                                                                    
PopularModel    0.000379  0.000055        3.637503  0.095908  0.055289   

                          recall            
                   std      mean       std  
model                                       
PopularModel  0.003696  0.076113  0.004111

In [24]:
movies_ml[movies_ml['movieId'].isin(model.recommend())]

,movieId,title,genres
314,318,"Shawshank Redemption, The (1994)",Crime|Drama
2480,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2867,2959,Fight Club (1999),Action|Crime|Drama|Thriller
4888,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
7029,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
12223,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
14939,79132,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX
21212,109487,Interstellar (2014),Sci-Fi|IMAX
57778,195159,Spider-Man: Into the Spider-Verse (2018),Action|Adventure|Animation|Sci-Fi
85911,286897,Spider-Man: Across the Spider-Verse (2023),Action|Adventure|Animation|Sci-Fi
